In [1]:
### Multilinear Regression Problem 


### Data bricks 

## Upload the dataset.
## Create the cluster ( 15 gb memory)
## You can use any kind of libraries ..
## Create a blank notebook which have the basic code.
## File location : 
## Then you can have the code

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MultipleLinearRegression").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/12 15:56:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/12 15:56:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
df = spark.read.csv("tips.csv", header = True , inferSchema = True)
df.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [8]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [9]:
df.columns

['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [ ]:
## My independent features is : tip , day 

In [10]:
## Handling categorical features :['sex', 'smoker', 'day', 'time']
## One way in pyspark : StringIndexer 
## Categorical feature to Ordinal encoding 
from pyspark.ml.feature import  StringIndexer

In [14]:
## How to convert ['sex', 'smoker', 'day', 'time'] into Numerical Data
indexer = StringIndexer(inputCol="sex",outputCol="sex_indexed")
df_r = indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|
+----------+----+------+------+---+------+----+-----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|        0.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|        0.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|        0.0|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|        0.0|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|        1.0|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        0.0|
|     18.43| 3.0|  Male|    No|Sun|Dinne

In [15]:
indexer = StringIndexer(inputCols=['smoker','day', 'time'],outputCols=["smoker_indexed", "day_indexed", "time_indexed"])
df_r = indexer.fit(df_r).transform(df_r)
df_r.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [16]:
## All the strings into numerical value now model will able to understand

## Vector Assembler 
## We need to group independent feature together and dependent together 

from pyspark.ml.feature import  VectorAssembler

In [17]:
df_r.columns

['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'smoker_indexed',
 'day_indexed',
 'time_indexed']

In [18]:
featureassembler = VectorAssembler(inputCols=['tip','size','sex_indexed','smoker_indexed','day_indexed','time_indexed'], outputCol="Independent Features ")

In [24]:
output = featureassembler.transform(df_r)

In [26]:
output.select('Independent Features ').show()

+---------------------+
|Independent Features |
+---------------------+
| [1.01,2.0,1.0,0.0...|
| [1.66,3.0,0.0,0.0...|
| [3.5,3.0,0.0,0.0,...|
| [3.31,2.0,0.0,0.0...|
| [3.61,4.0,1.0,0.0...|
| [4.71,4.0,0.0,0.0...|
| [2.0,2.0,0.0,0.0,...|
| [3.12,4.0,0.0,0.0...|
| [1.96,2.0,0.0,0.0...|
| [3.23,2.0,0.0,0.0...|
| [1.71,2.0,0.0,0.0...|
| [5.0,4.0,1.0,0.0,...|
| [1.57,2.0,0.0,0.0...|
| [3.0,4.0,0.0,0.0,...|
| [3.02,2.0,1.0,0.0...|
| [3.92,2.0,0.0,0.0...|
| [1.67,3.0,1.0,0.0...|
| [3.71,3.0,0.0,0.0...|
| [3.5,3.0,1.0,0.0,...|
| (6,[0,1],[3.35,3.0])|
+---------------------+
only showing top 20 rows



In [28]:
## 
output.show(5)

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+---------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|Independent Features |
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+---------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0| [1.01,2.0,1.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0| [1.66,3.0,0.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0| [3.5,3.0,0.0,0.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0| [3.31,2.0,0.0,0.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0| [3.61,4.0,1.0,0.0...|
+----------+----+------+

In [29]:
finalized_data = output.select('Independent Features ', 'total_bill')

In [30]:
finalized_data.show(5)

+---------------------+----------+
|Independent Features |total_bill|
+---------------------+----------+
| [1.01,2.0,1.0,0.0...|     16.99|
| [1.66,3.0,0.0,0.0...|     10.34|
| [3.5,3.0,0.0,0.0,...|     21.01|
| [3.31,2.0,0.0,0.0...|     23.68|
| [3.61,4.0,1.0,0.0...|     24.59|
+---------------------+----------+
only showing top 5 rows



In [31]:
## Applying Linear Regression 

from pyspark.ml.regression import  LinearRegression

In [33]:
## train test split 
train_data, test_data  = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Features ', labelCol='total_bill')
regressor=regressor.fit(train_data)

23/07/12 16:21:09 WARN Instrumentation: [f0fc9998] regParam is zero, which might cause numerical instability and overfitting.
23/07/12 16:21:11 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/07/12 16:21:11 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/07/12 16:21:11 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [35]:
regressor.coefficients

DenseVector([2.538, 3.6882, -1.1981, 1.4332, -0.3947, -1.4154])

In [36]:
regressor.intercept

3.3313793942478296

In [37]:
### Predictions 
pred_results = regressor.evaluate(test_data)

In [38]:
pred_results.predictions.show()

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+---------------------+----------+------------------+
|Independent Features |total_bill|        prediction|
+---------------------+----------+------------------+
| (6,[0,1],[1.25,2.0])|     10.07| 13.88022721033362|
| (6,[0,1],[1.45,2.0])|      9.55|14.387829307213794|
| (6,[0,1],[1.47,2.0])|     10.77| 14.43858951690181|
| (6,[0,1],[1.97,2.0])|     12.02|15.707594759102246|
|  (6,[0,1],[2.0,2.0])|     13.37|15.783735073634272|
| (6,[0,1],[2.24,3.0])|     16.04|20.081024945182836|
| (6,[0,1],[2.31,3.0])|     18.69|20.258685679090895|
| (6,[0,1],[2.34,4.0])|     17.81| 24.02299334891527|
|  (6,[0,1],[2.5,4.0])|     18.35| 24.42907502641941|
|  (6,[0,1],[3.0,2.0])|      14.0| 18.32174555803514|
|  (6,[0,1],[3.0,4.0])|     20.45|25.698080268619844|
| (6,[0,1],[3.15,3.0])|     20.08|22.390614485987623|
| (6,[0,1],[3.39,2.0])|     11.61| 19.31156964695148|
|  (6,[0,1],[5.0,3.0])|     31.27|27.085933882129233|
| (6,[0,1],[6.73,4.0])|     48.27| 35.16485937543509|
| [1.25,2.0,1.0,0.0...|     

In [39]:

pred_results.r2

0.6254824091955886

In [40]:
pred_results.meanAbsoluteError

5.021144677012349

In [41]:
pred_results.meanSquaredError

41.24595588973175

In [ ]:
## How you can save the file in the pickle format. 